In [1]:
import pandas as pd 
import numpy as np
import urllib.request
url = "https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin"
urllib.request.urlretrieve(url, "pipeline_v1.bin")

('pipeline_v1.bin', <http.client.HTTPMessage at 0x1d2ce60b950>)

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline

In [9]:
import pickle
with open("pipeline_v1.bin", "rb") as f:
    pipeline = pickle.load(f)

C:\Users\aliel\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\aliel\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\aliel\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.5.1. This might lead to breaking c

In [11]:
record = {
    "lead_score":"paid_ads",
    "number_of_courses_viewed":2,
    "annual_income":79276.0
}

prob = pipeline.predict_proba([record])[0,1]
print(prob)

0.533607270279322


In [16]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float


@app.post("/predict")
def predict(client : Client):
    client_dict = client.dict()
    prob =pipeline.predict_proba([client_dict])[0,1]
    return {"probability": float(prob)}

In [26]:
import requests
url = "http://127.0.0.1:8000/predict"

client = {
    "lead_score": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income":80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'probability': 0.534}


In [1]:
import requests
url = "http://127.0.0.1:8000/predict"

client = {
    "lead_score": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income":80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'probability': 0.534}
